# Generate Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
import json
sys.path.append('../src')

import requests
from io import StringIO
import time

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### S&P 500 

In [16]:
tickers = ['^GSPC']
start = '852076800'
base = 'https://finance.yahoo.com/quote/'
base2 = 'https://query1.finance.yahoo.com/v7/finance/download/'
end = int(time.time())
dates = '?period1={}&period2={}'.format(start, end)
param = '&interval=1d&filter=history&frequency=1d'

d = {}
for t in tickers:
    url = '{}{}/history{}{}'.format(base, t, dates, param)
    s = requests.Session()
    r = s.get(url)
    regex = '"CrumbStore":{"crumb":"(.+?)"},'
    pattern = re.compile(regex)
    crumb = re.findall(pattern, r.text)[0]
    param2 = '{}{}&interval=1d&events=history&crumb={}/Q'.format(
        t, dates, crumb)
    url2 = '{}{}'.format(base2, param2)
    data = s.post(url2)
    csv = StringIO(data.text)
    d[t[1:]] = pd.read_table(csv, sep=',',
                             parse_dates=['Date']).set_index('Date')['Adj Close']
df = pd.DataFrame(d)
final = df.resample('MS').mean().append(df.iloc[-1])

final.to_csv(data_dir / 'sp500.csv', index_label='date', float_format='%g')

ValueError: 'Date' is not in list

### Interest rates

In [5]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H15&series=398c5ba6279c57e57055c6a65c95e9d3&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d = {
    'RIFLGFCY10_N.B': 'Ten-year',
    'RIFSPFF_N.B': 'Fed Funds',
    'RIFLGFCM03_N.B': 'Three-month',
    'RIFLGFCY02_N.B': 'Two-year'
}

df = pd.read_csv(url, skiprows=5, index_col=0)[d.keys()].rename(d, axis=1)

df.index = pd.to_datetime(df.index)

data = (df[df['Ten-year'] != 'ND'].astype('float')
        .resample('M').mean().iloc[:-1]
        .append(df.iloc[-1]))

data.to_csv(data_dir / 'rates.csv', index_label='date', float_format='%g')

ldate = dtxt(data.index[-1])['day1']
tenlt = data['Ten-year'].iloc[-1]
tenpr = data['Ten-year'].iloc[-13]
twolt = data['Two-year'].iloc[-1]
twopr = data['Two-year'].iloc[-13]

text = (f'As of {ldate}, the constant maturity yield for a ten-year '+
        'Treasury bond (see {\color{red!70!purple!90!black}\\textbf{---}}) '+
        f'is {tenlt} percent, compared to {tenpr:.2f} percent one-year prior. '+
        'The yield for a two-year Treasury (see {\color{orange!80!yellow}\\textbf{---}}) '
        +f'is {twolt} percent, compared to {twopr:.2f} '+
        f'percent a year prior.')

write_txt(text_dir / 'rates_basic.txt', text)

print(text)

text = ('The effective fed funds (see {\color{blue!60!black}\\textbf{---}}) '+
        f'rate is {data["Fed Funds"].iloc[-1]} percent, as of {ldate}.')

write_txt(text_dir / 'rates_ff.txt', text)

text

As of March 2, 2020, the constant maturity yield for a ten-year Treasury bond (see {\color{red!70!purple!90!black}\textbf{---}}) is 1.10 percent, compared to 2.57 percent one-year prior. The yield for a two-year Treasury (see {\color{orange!80!yellow}\textbf{---}}) is 0.84 percent, compared to 2.41 percent a year prior.


'The effective fed funds (see {\\color{blue!60!black}\\textbf{---}}) rate is 1.59 percent, as of March 2, 2020.'

### Exchange rates

In [6]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H10&series=ad1712193ad5bad7b424e3ae5eb101a5&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

clean_data['RXI_N.B.EU'] = 1 / clean_data['RXI$US_N.B.EU'] 
clean_data['RXI_N.B.UK'] = 1 / clean_data['RXI$US_N.B.UK'] 
clean_data['RXI_N.B.JA'] = clean_data['RXI_N.B.JA'] / 100.0

latest = clean_data.iloc[-1]
major = ['RXI_N.B.EU', 'RXI_N.B.UK', 'RXI_N.B.CA', 'RXI_N.B.JA']
oth = ['RXI_N.B.MX', 'RXI_N.B.BZ', 'RXI_N.B.CH', 'RXI_N.B.SD']
twidx = clean_data.resample('MS').mean().append(latest)['JRXWTFB_N.B'].dropna()
clean_data.resample('MS').mean().append(latest)[major].to_csv(data_dir / 'fx1.csv', index_label='date', float_format='%g')
clean_data.resample('MS').mean().append(latest)[oth].to_csv(data_dir / 'fx2.csv', index_label='date', float_format='%g')
twidx.to_csv(data_dir / 'fx_idx.csv', index_label='date', float_format='%g', header=True)

ldate = dtxt(twidx.index[-1])['day1']
lval = twidx.iloc[-1]
totch = ((lval / 100) - 1) * 100
threeyr = twidx.iloc[-38:].mean()
prev3yr = twidx.iloc[-74:-38].mean()

text = (f'The latest index value, as of {ldate}, is {lval:.1f}, thus an increase '+
        f'of {totch:.1f} percent since inception in 2006. Over the past three years, '+
        f'the index value has averaged {threeyr:.1f}, compared to an average '+
        f'of {prev3yr:.1f} over the previous three year period.')

write_txt(text_dir / 'twdbasic.txt', text)

text

'The latest index value, as of February 28, 2020, is 117.8, thus an increase of 17.8 percent since inception in 2006. Over the past three years, the index value has averaged 113.6, compared to an average of 106.3 over the previous three year period.'

In [7]:
cad = clean_data.loc[ldate, 'RXI_N.B.CA']
jpy = clean_data.loc[ldate, 'RXI_N.B.JA'] * 100
eur = clean_data.loc[ldate, 'RXI_N.B.EU']
gbp = clean_data.loc[ldate, 'RXI_N.B.UK']
cadpc = inc_dec_percent(clean_data['RXI_N.B.CA'].pct_change(262).iloc[-1] * 100)
jpypc = inc_dec_percent(clean_data['RXI_N.B.JA'].pct_change(262).iloc[-1] * 100)
gbppc = inc_dec_percent(clean_data['RXI_N.B.UK'].pct_change(262).iloc[-1] * 100)
eurpc = inc_dec_percent(clean_data['RXI_N.B.EU'].pct_change(262).iloc[-1] * 100)


text = (f'As of {ldate}, one US dollar buys approximately: {cad:.2f} Canadian dollars '+
        f'(see {{\color{{green!85!blue}}\\textbf{{---}}}}), {jpy:.0f} Japanese Yen '+
        f'(see {{\color{{red}}\\textbf{{---}}}}), {eur:.2f} Euros '+
        f'(see {{\color{{cyan!90!white}}\\textbf{{---}}}}), and {gbp:.2f} British Pounds '+
         '(see {\color{blue!90!cyan}\\textbf{---}}). Over the past three years, the nominal '+
        f'exchange rate between the US dollar and the Canadian dollar {cadpc}, the USD-JPY rate '+
        f'{jpypc}, the USD-EUR rate {eurpc}, and the USD-GBP rate {gbppc}.')

write_txt(text_dir / 'selcurr_basic.txt', text)

text

'As of February 28, 2020, one US dollar buys approximately: 1.34 Canadian dollars (see {\\color{green!85!blue}\\textbf{---}}), 108 Japanese Yen (see {\\color{red}\\textbf{---}}), 0.91 Euros (see {\\color{cyan!90!white}\\textbf{---}}), and 0.78 British Pounds (see {\\color{blue!90!cyan}\\textbf{---}}). Over the past three years, the nominal exchange rate between the US dollar and the Canadian dollar increased by 2.0 percent, the USD-JPY rate decreased by 2.5 percent, the USD-EUR rate increased by 3.4 percent, and the USD-GBP rate increased by 4.2 percent.'

### Initial jobless claims

In [8]:
series = 'ICSA'
start = '1988-01-01'
base = 'https://api.stlouisfed.org/fred/series/observations?'
param = f'series_id={series}&observation_start={start}&api_key={fred_key}'
ftype = '&file_type=json'

url = f'{base}{param}{ftype}'
r = requests.get(url).json()['observations']
data = pd.DataFrame(pd.Series({i['date']: int(i['value']) / 1000 
                               for i in r}).rename('weekly'))

data['3M'] = data['weekly'].rolling(12).mean()

data.to_csv(data_dir / 'icsa.csv', index_label='date', float_format='%g')

totval = data['weekly'].iloc[-1]*1000
datelt = dtxt(data.index[-1])['day1']
latest3m = data["3M"].iloc[-1]*1000
prev3m = data["3M"].iloc[-157]*1000

text = ('The Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{reported}} '+
        f'{totval:,.0f} initial claims for unemployment '+
        f'insurance during the week ending {datelt}. Over the past three months, '+
        f'initial claims averaged {latest3m:,.0f} per week. During the same three month period '+
        f'three years ago, initial claims averaged {prev3m:,.0f} per week.')

write_txt(text_dir / 'icsa.txt', text)

### VIX (SP500 volatility)

In [9]:
prev = pd.read_excel(data_dir/ 'vixarchive.xls', skiprows=1, index_col='Date')
VIX = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv'
curr = pd.read_csv(VIX, skiprows=1, index_col='Date', parse_dates=True)
df = (prev.append(curr)['VIX Close'].resample('MS').mean()
      .append(curr['VIX Close'].iloc[-1:]).rename('value'))
df.to_csv(data_dir / 'vix.csv', index_label='date', header='True')

ldate = dtxt(df.index[-1])['day1']

text = (f'This volatility measure, VIX, was {df.iloc[-1]:.1f} on {ldate}, '+
        f'compared to an average of {df.iloc[-37:].mean():.1f} over the '+
        'past three years.')

write_txt(text_dir / 'vixbasic.txt', text)

text

'This volatility measure, VIX, was 36.8 on March 3, 2020, compared to an average of 15.9 over the past three years.'

### Oil prices (WTI)

In [10]:
df = fred_df('DCOILWTICO')

latest = df.iloc[-1]

ma = df.resample('MS').mean()

p = ma.append(latest)['VALUE']

(p.to_csv(data_dir / 'wti.csv', index_label='date', header=True))

oneyr = p.pct_change(13).iloc[-1] * 100

threeyr = p.pct_change(37).iloc[-1] * 100

oyt = inc_dec_percent(oneyr)
tyt = inc_dec_percent(threeyr)
    
ltch = p.loc['2008-06-01'] - p.iloc[-1]

ldate = dtxt(p.index[-1])['day1']

text = (f'As of {ldate}, a barrel of west Texas '+
        f'intermediate (WTI) \textbf{{crude oil}} sells for \${p.iloc[-1]}. Over the '+
        f'past year, this measure of oil prices has {oyt}. Over the past three '+
        f'years, the price {tyt}. Currently, the WTI price is \${ltch:.2f} per barrel '+
        'below its June 2008 average.')

write_txt(text_dir / 'wti.txt', text)

text

'As of February 24, 2020, a barrel of west Texas intermediate (WTI) \textbf{crude oil} sells for \\$51.36. Over the past year, this measure of oil prices has decreased by 6.5 percent. Over the past three years, the price decreased by 3.9 percent. Currently, the WTI price is \\$82.52 per barrel below its June 2008 average.'

### Inflation Expectations

In [11]:
url = 'http://www.sca.isr.umich.edu/files/tbmpx1px5.csv'
df = pd.read_csv(url)
df.index = pd.to_datetime(df['Month'] + ' ' + df['YYYY'].astype('str'))
data = df['PX5_MD'].loc['2015':]

data.to_csv(data_dir / 'infumich.csv', index_label='date', header='VALUE')

node = end_node(data, 'violet')
write_txt(text_dir / 'infumich_node.txt', node)

data1 = fred_df('T5YIE').loc['2015':,'VALUE']
data1.to_csv(data_dir / 'infbreak.csv', index_label='date', header=True)

node = end_node(data1, 'blue!70!black')
write_txt(text_dir / 'infbreak_node.txt', node)

In [12]:
ldate = dtxt(data.index[-1])['mon1']
pdate = dtxt(data.index[-13])['mon1']
p5val = data.iloc[-61]
lval = data.iloc[-1]
pval = data.iloc[-13]

ldatem = dtxt(data1.index[-1])['day1']
lvalm = data1.iloc[-1]
pdatem = dtxt(data1.index[-262])['day1']
pvalm = data1.iloc[-262]
p5valm = data1.iloc[-(262*5)]

inf_act = pd.read_csv(data_dir / 'cpi.csv')['ALL'].iloc[-60:].mean()

text = (f'As of {ldate}, consumers expect an average inflation rate of {lval} '+
        'percent over the next five years, (see {\color{violet!60!magenta}\\textbf{---}}), '+
        f'compared to an expected rate of {pval} percent '+
        f'on {pdate}. Consumers had expected inflation to average {p5val} percent over the past '+
        f'five years, while actual inflation over the period was {inf_act:.1f} percent.')

write_txt(text_dir / 'inf_exp_cons.txt', text)

text

'As of February 2020, consumers expect an average inflation rate of 2.3 percent over the next five years, (see {\\color{violet!60!magenta}\\textbf{---}}), compared to an expected rate of 2.3 percent on February 2019. Consumers had expected inflation to average 2.7 percent over the past five years, while actual inflation over the period was 1.6 percent.'

In [13]:
text2 = (f'As of {ldatem}, markets expect an average inflation rate of {lval} percent ' + 
         'over the next five years (see {\color{blue!70!black}\\textbf{---}}), '
         +f'compared to an expected rate of {pvalm} percent on '+
         f'{pdatem}. Markets had expected inflation to average {p5valm} over the past five '+
         f'years, five years ago.')

write_txt(text_dir / 'inf_exp_mkts.txt', text2)

text2

'As of March 3, 2020, markets expect an average inflation rate of 2.3 percent over the next five years (see {\\color{blue!70!black}\\textbf{---}}), compared to an expected rate of 1.84 percent on March 4, 2019. Markets had expected inflation to average 1.36 over the past five years, five years ago.'